In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/womens-e-commerce-clothing-reviews/Women_s_E-Commerce_Clothing_Reviews_1594_1.csv


**Read CSV files with semicolon separator**

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from scipy.stats import pearsonr

# 1. Membaca file CSV dengan pemisah titik koma
df = pd.read_csv('/kaggle/input/womens-e-commerce-clothing-reviews/Women_s_E-Commerce_Clothing_Reviews_1594_1.csv', sep=';', on_bad_lines='skip')

**Data preview.**

**Clean data (delete rows with missing values ​​in 'Review.Text' and 'Rating' columns).**

In [3]:
# 2. Pratinjau data
print(df.head())

# 3. Bersihkan data (menghapus baris dengan nilai yang hilang di kolom 'Review.Text', 'Title', dan 'Rating')
df.dropna(subset=['Review.Text', 'Title', 'Rating'], inplace=True)

   Unnamed: 0  Clothing.ID  Age                    Title  \
0           1          767   33                      NaN   
1           2         1080   34                      NaN   
2           3         1077   60  Some major design flaws   
3           4         1049   50         My favorite buy!   
4           5          847   47         Flattering shirt   

                                         Review.Text  Rating  Recommended.IND  \
0  Absolutely wonderful - silky and sexy and comf...       4                1   
1  Love this dress!  it's sooo pretty.  i happene...       5                1   
2  I had such high hopes for this dress and reall...       3                0   
3  I love, love, love this jumpsuit. it's fun, fl...       5                1   
4  This shirt is very flattering to all due to th...       5                1   

   Positive.Feedback.Count   Division.Name Department.Name Class.Name  
0                        0       Initmates        Intimate  Intimates  
1       

**Text processing**

In [4]:
# 4. Pemrosesan teks
X_review = df['Review.Text']
y_review = df['Rating']

**Categorize ratings into 3 classes: negative (1-2), neutral (3), positive (4-5)**

In [5]:
# Mengkategorikan rating menjadi 3 kelas: negatif (1-2), netral (3), positif (4-5)
y_review = y_review.apply(lambda x: 'positive' if x > 3 else ('negative' if x < 3 else 'neutral'))

**Feature extraction using TF-IDF**

In [6]:
# 5. Ekstraksi fitur menggunakan TF-IDF untuk 'Review.Text'
vectorizer_review = TfidfVectorizer(stop_words='english', max_features=5000)
X_review_tfidf = vectorizer_review.fit_transform(X_review)

**Split data into training and test sets**

In [7]:
# 6. Membagi data menjadi set pelatihan dan pengujian untuk 'Review.Text'
X_review_train, X_review_test, y_review_train, y_review_test = train_test_split(X_review_tfidf, y_review, test_size=0.2, random_state=42)

**Training model classification**

In [8]:
# 7. Melatih model klasifikasi untuk 'Review.Text'
model_review = LogisticRegression()
model_review.fit(X_review_train, y_review_train)

/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

**Model evaluation**

In [9]:
# 8. Evaluasi model untuk 'Review.Text'
y_review_pred = model_review.predict(X_review_test)
print(f'Accuracy for Review.Text: {accuracy_score(y_review_test, y_review_pred)}')
print(classification_report(y_review_test, y_review_pred))

Accuracy for Review.Text: 0.8269377382465057
              precision    recall  f1-score   support

    negative       0.66      0.39      0.49       438
     neutral       0.42      0.26      0.32       458
    positive       0.87      0.98      0.92      3039

    accuracy                           0.83      3935
   macro avg       0.65      0.54      0.58      3935
weighted avg       0.80      0.83      0.80      3935



**Added a sentiment prediction column to the original DataFrame**

In [10]:
# 9. Prediksi sentimen untuk 'Review.Text'
df['Review.Sentiment'] = df['Review.Text'].apply(lambda text: model_review.predict(vectorizer_review.transform([text]))[0])

**Text processing for "title"**

In [11]:
# 10. Pemrosesan teks untuk 'Title'
X_title = df['Title']
y_title = df['Rating']

**Categorize ratings into 3 classes: negative (1-2), neutral (3), positive (4-5)**

In [12]:
y_title = y_title.apply(lambda x: 'positive' if x > 3 else ('negative' if x < 3 else 'neutral'))

**Feature extraction using TF-IDF for 'Title'**

In [13]:
vectorizer_title = TfidfVectorizer(stop_words='english', max_features=5000)
X_title_tfidf = vectorizer_title.fit_transform(X_title)

**Splitting data into training and test sets for 'Title'**

In [14]:
X_title_train, X_title_test, y_title_train, y_title_test = train_test_split(X_title_tfidf, y_title, test_size=0.2, random_state=42)

**Training a classification model for 'Title'**

In [15]:
model_title = LogisticRegression()
model_title.fit(X_title_train, y_title_train)

/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

**Model evaluation for 'Title'**

In [16]:
y_title_pred = model_title.predict(X_title_test)
print(f'Accuracy for Title: {accuracy_score(y_title_test, y_title_pred)}')
print(classification_report(y_title_test, y_title_pred))

Accuracy for Title: 0.8213468869123253
              precision    recall  f1-score   support

    negative       0.64      0.39      0.48       438
     neutral       0.41      0.19      0.26       458
    positive       0.86      0.98      0.92      3039

    accuracy                           0.82      3935
   macro avg       0.64      0.52      0.55      3935
weighted avg       0.78      0.82      0.79      3935



**Sentiment predictions for 'Title'**

In [17]:
df['Title.Sentiment'] = df['Title'].apply(lambda text: model_title.predict(vectorizer_title.transform([text]))[0])

**Convert sentiment to numeric values ​​for correlation**

In [18]:
sentiment_mapping = {'negative': -1, 'neutral': 0, 'positive': 1}
df['Review.Sentiment.Numeric'] = df['Review.Sentiment'].map(sentiment_mapping)
df['Title.Sentiment.Numeric'] = df['Title.Sentiment'].map(sentiment_mapping)

**Removes rows with NaN values ​​in columns 'Review.Sentiment.Numeric' and 'Title.Sentiment.Numeric'**

In [19]:
df.dropna(subset=['Review.Sentiment.Numeric', 'Title.Sentiment.Numeric'], inplace=True)

**Calculating the Pearson correlation between 'Review.Text' and 'Title' sentiment**

In [20]:
correlation, _ = pearsonr(df['Review.Sentiment.Numeric'], df['Title.Sentiment.Numeric'])
print(f'Korelasi antara sentimen Title dan Review.Text: {correlation}')

Korelasi antara sentimen Title dan Review.Text: 0.4934313449421262


**Save the DataFrame with the prediction column to a new file**

In [21]:
df.to_csv('Women_s_E-Commerce_Clothing_Reviews_with_Sentiment.csv', sep=';', index=False)

print("Hasil prediksi sentimen telah disimpan ke 'Women_s_E-Commerce_Clothing_Reviews_with_Sentiment.csv'.")

Hasil prediksi sentimen telah disimpan ke 'Women_s_E-Commerce_Clothing_Reviews_with_Sentiment.csv'.
